In [13]:
import pandas as pd

df = pd.read_json("vndb-tags-2023-01-10.json")

split = []
do_splitting = True

def change_tag(row):
    global split, do_splitting
    a = type(row)
    if len(row["parents"]) == 0:
        row["parents"] = 0
    elif len(row["parents"]) == 1:
        row["parents"] = row["parents"][0]
    elif len(row["parents"]) > 1:
        if do_splitting:
            row["parents"] = row["parents"][0]
            for parent in row["parents"][1:]:
                row1 = pd.DataFrame(columns=df.columns, data=[[row["aliases"], row["applicable"], row["cat"], row["description"], row["id"], row["meta"], row["name"], parent, row["searchable"], row["vns"]]])
                split.append(row1)
        else:
            row["parents"] = row["parents"][0]
            
    return row
    


df = df.apply(change_tag, axis=1)

root = pd.DataFrame(columns=df.columns, data=[['', False, "tech", "", 0, True, "Tags", None, False, 0]])

if do_splitting:
    split_pd = pd.concat(split, axis=0)
    df = pd.concat([df,split_pd, root], axis=0, ignore_index=True)
else:
    df = pd.concat([df, root], axis=0, ignore_index=True)

In [11]:
import plotly.graph_objects as go


fig = go.Figure(go.Treemap(
    ids= df["id"],
    labels=df["name"],
    parents=df["parents"],
    values=df["vns"],
    root_color="lightgrey"
))

fig.update_layout(
    margin = dict(t=50, l=25, r=25, b=25)
)
fig.show()
# fig.write_html("index.html")